In [1]:
from pydantic import BaseModel, field_validator, ValidationInfo

In [2]:
class Model(BaseModel):
    field_1: int
    field_2: list[int]
    field_3: str
    field_4: list[str]
    
    @field_validator("field_3")
    def validator(cls, value: str, validated_values: ValidationInfo) -> str:
        print(f"{value=}")
        print(f"{validated_values=}")
        return value

In [3]:
Model(field_1=1, field_2=[1], field_3="a", field_4=['a'])

value='a'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 1, 'field_2': [1]}, field_name='field_3')


Model(field_1=1, field_2=[1], field_3='a', field_4=['a'])

In [4]:
Model(field_1=100, field_2=["a"], field_3="a", field_4=['a'])

value='a'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100}, field_name='field_3')


ValidationError: 1 validation error for Model
field_2.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

In [5]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse
from pydantic import AfterValidator, BeforeValidator


def parse_datetime(value: Any):
    if isinstance(value, str):
        try: 
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [6]:
class Model(BaseModel):
    start_dt: DateTimeUTC
    end_dt: DateTimeUTC
    
    @field_validator("end_dt")
    def validate_end_after_start_date(
            cls, 
            value: datetime, 
            values: ValidationInfo
    ) -> datetime:
        date: dict = values.data
        if "start_dt" in date:
            if value <= date['start_dt']:
                raise ValueError('end_dt must come after start_dt')
        return value  

In [7]:
Model(start_dt="2020/1/1", end_dt="2020/12/31")

Model(start_dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), end_dt=datetime.datetime(2020, 12, 31, 0, 0, tzinfo=<UTC>))

In [10]:
Model(start_dt="2020/1/1", end_dt="2020/1/1")

ValidationError: 1 validation error for Model
end_dt
  Value error, end_dt must come after start_dt [type=value_error, input_value='2020/1/1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error